# Notebook 3: Funções de Transformação e Limpeza de Dados

## Introdução
Neste notebook, iremos implementar as transformações e limpar os dados da tabela 'Outcome' do conjunto de dados FAERS, com base na análise exploratória realizada no notebook anterior (Exploratory_Data_Analysis_Outcome). 

## Estrutura do notebook
- Importação de Bibliotecas
- Carregamento dos Dados (Parquet Raw) usando o Schema definido 
- Aplicar apenas as Transformações necessárias
- Salvar **Parquet Final** com os Dados Limpos


In [0]:
# Importar bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, LongType
from pyspark.sql.functions import col

In [0]:
%run "../Funcoes_auxiliares" 

In [0]:
# Carregar dados do Parquet raw
schema_outc = StructType([
    StructField("primaryid", LongType(), False,{'description': "Unique number for identifying a FAERS report."}),
    StructField("caseid", IntegerType(), False,{'description': "Number for identifying a FAERS case."}),
    StructField("outc_cod", StringType(), False, {'description': "Code for a patient outcome."})
    ])
df_outc = spark.read.format('parquet').schema(schema_outc).load('dbfs:/FileStore/FAERS-grupo-4/outc_raw')



In [0]:
# Criar DataFrame de mapeamento de códigos para descrições
mapping_data = [
    ("DE", "Death"),
    ("LT", "Life Threatening"),
    ("HO", "Hospitalization - Initial or Prolonged"),
    ("DS", "Disability"),
    ("CA", "Congenital Anomaly"),
    ("RI", "Required Intervention to Prevent Permanent Impairment/Damage"),
    ("OT", "Other Serious (Important Medical Event)")
]
mapping_columns = ["outc_cod", "outcome_name"]
df_mapping = spark.createDataFrame(mapping_data, mapping_columns)

# Join entre df_outc e o DataFrame de mapeamento
df_outc = df_outc.join(df_mapping, on="outc_cod", how="left")['primaryid','caseid','outc_cod','outcome_name']


In [0]:
#Restaurar metadados
df_outc = restore_metadata(schema_outc,df_outc)

In [0]:
# salvar parquet final
df_outc.write.mode('overwrite').parquet("dbfs:/FileStore/FAERS-grupo-4/outc_final")